## 🖥️ [로컬 실행용] KcELECTRA 악성댓글 분류 모델 학습

> ⚠️ **이 노트북은 로컬 환경(Windows + GPU)용으로 수정되었습니다.**  
> Google Colab에서 실행하지 마세요!

### 실행 전 필수 사항
1. **Anaconda 환경 활성화**: `conda activate kcelectra`
2. **데이터 파일 확인**: `C:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\최종 데이터셋` 폴더에 CSV 파일이 있어야 합니다.
3. **첫 번째 셀(패키지 설치)을 반드시 먼저 실행**하세요.

---

### 1️⃣ 패키지 설치 (최초 1회만 실행)

In [5]:
# ========================================
# 📦 패키지 설치 (최초 1회만 실행)
# ========================================
# 이미 설치했다면 이 셀은 건너뛰어도 됩니다.

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas
%pip install -U datasets

print("✅ 패키지 설치 완료!")

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ 패키지 설치 완료!


In [6]:
# ========================================
# 🔧 로컬 경로 설정 (본인 환경에 맞게 수정)
# ========================================
import os

# 절대 경로 사용 (본인 환경에 맞게 수정하세요)
DATA_DIR = r"c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data"
OUTPUT_DIR = r"./kcelectra_output"  # 모델 저장 경로 (현재 폴더 기준)

# 경로 확인
if not os.path.exists(DATA_DIR):
    print(f"⚠️ 데이터 폴더를 찾을 수 없습니다: {DATA_DIR}")
    print(f"   현재 작업 디렉토리: {os.getcwd()}")
else:
    print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 폴더: c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data
📁 출력 폴더: ./kcelectra_output


---
### 2️⃣ KcELECTRA 모델 학습

✔ Train으로 학습

✔ Test 셋은 학습에 쓰지 않음

✔ Test 셋은 Trainer의 evaluate()에 연결

✔ 자동으로 F1/Acc 등 출력

In [7]:
# 1) 환경 준비
import os
import random
import numpy as np
import torch

# 랜덤 시드 고정 → 매번 동일한 결과 재현 가능
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU가 감지되지 않았습니다. CPU로 실행됩니다.")

✅ GPU 사용 가능: NVIDIA GeForce RTX 4060 Ti


In [8]:
# 2) 데이터 불러오기 + 합치기
import pandas as pd

# 로컬 데이터 경로 설정 (datasets 폴더 기준 파일명)
abusive_path = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
nonabusive_path = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")

# 파일 존재 확인
if not os.path.exists(abusive_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {abusive_path}")
if not os.path.exists(nonabusive_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {nonabusive_path}")

# 두 csv를 불러와 text, label만 선택
df_ab = pd.read_csv(abusive_path)[["text", "label"]]
df_non = pd.read_csv(nonabusive_path)[["text", "label"]]

# 데이터 합치기
df = pd.concat([df_ab, df_non], ignore_index=True)
df = df.dropna().reset_index(drop=True)

print(df.head())
print(f"\n📊 라벨 분포:\n{df['label'].value_counts()}")
print(f"\n총 데이터 수: {len(df)}개")

                                                text  label
0                                            ᄂᄀᄀᄈ 꺼져      1
1                                                씹퇭아      1
2                           누군지모르지만 요즘걸그룹창여들다발라버리네..      0
3  자살하면 다음생 좋은데 갈꺼같냐ㅋㅋ?절때 못가지금수저로는 더더욱 못태어남100프로흙...      1
4                                  니좆이*다 니좆이*다 니좆이*다      1

📊 라벨 분포:
label
0    22882
1    17118
Name: count, dtype: int64

총 데이터 수: 40000개


In [9]:
# 3) Train / Test 분리
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]    # 라벨 비율 유지
)

print(f"Train size: {len(train_df)}")
print(f"Test size : {len(test_df)}")

Train size: 32000
Test size : 8000


In [10]:
# 4) DatasetDict 구성 (train + validation=test)
from datasets import Dataset, DatasetDict

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

print(raw_ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 32000
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 8000
    })
})


In [11]:
# 5) 토크나이저 로드 + 토큰화
from transformers import AutoTokenizer

MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 토큰화 함수
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# DatasetDict에 토큰화 적용
tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")  # Trainer 규약에 맞게 labels로 변경
tokenized.set_format("torch")

print("✅ 토큰화 완료!")
print(tokenized)

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

✅ 토큰화 완료!
DatasetDict({
    train: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32000
    })
    validation: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
})


In [12]:
# 6) 클래스 가중치 계산 (데이터 불균형 보정)
import torch.nn as nn

labels_np = train_df["label"].values
pos = (labels_np == 1).sum()
neg = (labels_np == 0).sum()

# 단순 역수 기반 클래스 가중치
w0 = 1 / neg
w1 = 1 / pos
scale = 2 / (w0 + w1)
class_weights = torch.tensor([w0 * scale, w1 * scale], dtype=torch.float)

print(f"Class weights: {class_weights}")

Class weights: tensor([0.8559, 1.1441])


In [13]:
# 7) 모델 로드 + Trainer 정의 (evaluate 활성화)
# 실행하면 Some weights of ElectraForSequenceClassification were not initialized~~ 나오는데 무시해도 됩니다
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except Exception:
        auc = float("nan")

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2
)

# Weighted loss 적용되는 Trainer 정의
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Trainer 설정 (하이퍼파라미터는 여기에)
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",         # epoch마다 test셋으로 evaluate() 실행
    save_strategy="epoch",         # epoch마다 checkpoint 저장
    load_best_model_at_end=True,   # 가장 F1 좋은 모델 자동 로드
    metric_for_best_model="f1",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],  # test셋을 validation으로 사용 → evaluate 가능
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Trainer 설정 완료!


In [14]:
# 8) 학습 수행 (train + evaluate 자동 출력)
trainer.train()

  0%|          | 0/3000 [00:00<?, ?it/s]

{'loss': 0.6839, 'grad_norm': 2.425734519958496, 'learning_rate': 4.9e-06, 'epoch': 0.05}
{'loss': 0.5756, 'grad_norm': 4.3658647537231445, 'learning_rate': 9.9e-06, 'epoch': 0.1}
{'loss': 0.4256, 'grad_norm': 4.2172532081604, 'learning_rate': 1.47e-05, 'epoch': 0.15}
{'loss': 0.33, 'grad_norm': 2.977836847305298, 'learning_rate': 1.9699999999999998e-05, 'epoch': 0.2}
{'loss': 0.2531, 'grad_norm': 6.806024551391602, 'learning_rate': 2.47e-05, 'epoch': 0.25}
{'loss': 0.2753, 'grad_norm': 5.396688461303711, 'learning_rate': 2.97e-05, 'epoch': 0.3}
{'loss': 0.2796, 'grad_norm': 3.0265049934387207, 'learning_rate': 2.9477777777777776e-05, 'epoch': 0.35}
{'loss': 0.2442, 'grad_norm': 4.198986530303955, 'learning_rate': 2.8922222222222224e-05, 'epoch': 0.4}
{'loss': 0.2438, 'grad_norm': 6.425168991088867, 'learning_rate': 2.8366666666666667e-05, 'epoch': 0.45}
{'loss': 0.2502, 'grad_norm': 1.1815909147262573, 'learning_rate': 2.781111111111111e-05, 'epoch': 0.5}
{'loss': 0.2291, 'grad_norm':

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.1905481219291687, 'eval_accuracy': 0.92275, 'eval_precision': 0.9190561529271206, 'eval_recall': 0.8986565420560748, 'eval_f1': 0.9087418783225044, 'eval_roc_auc': 0.9789849506874552, 'eval_runtime': 16.6454, 'eval_samples_per_second': 480.613, 'eval_steps_per_second': 15.019, 'epoch': 1.0}
{'loss': 0.14, 'grad_norm': 3.675835371017456, 'learning_rate': 2.1711111111111112e-05, 'epoch': 1.05}
{'loss': 0.1326, 'grad_norm': 3.6549899578094482, 'learning_rate': 2.116666666666667e-05, 'epoch': 1.1}
{'loss': 0.1524, 'grad_norm': 2.2765233516693115, 'learning_rate': 2.0611111111111112e-05, 'epoch': 1.15}
{'loss': 0.1476, 'grad_norm': 6.985886096954346, 'learning_rate': 2.0055555555555556e-05, 'epoch': 1.2}
{'loss': 0.1488, 'grad_norm': 3.094674587249756, 'learning_rate': 1.95e-05, 'epoch': 1.25}
{'loss': 0.1359, 'grad_norm': 4.245350360870361, 'learning_rate': 1.8944444444444447e-05, 'epoch': 1.3}
{'loss': 0.145, 'grad_norm': 1.7562111616134644, 'learning_rate': 1.838888888888

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.20958544313907623, 'eval_accuracy': 0.92825, 'eval_precision': 0.9320800485142511, 'eval_recall': 0.8977803738317757, 'eval_f1': 0.9146087473966081, 'eval_roc_auc': 0.9809351078973597, 'eval_runtime': 16.6636, 'eval_samples_per_second': 480.089, 'eval_steps_per_second': 15.003, 'epoch': 2.0}
{'loss': 0.0682, 'grad_norm': 4.678597927093506, 'learning_rate': 1.0611111111111111e-05, 'epoch': 2.05}
{'loss': 0.0723, 'grad_norm': 6.029900074005127, 'learning_rate': 1.0055555555555557e-05, 'epoch': 2.1}
{'loss': 0.0735, 'grad_norm': 2.24930477142334, 'learning_rate': 9.5e-06, 'epoch': 2.15}
{'loss': 0.0812, 'grad_norm': 3.4027059078216553, 'learning_rate': 8.944444444444444e-06, 'epoch': 2.2}
{'loss': 0.0845, 'grad_norm': 6.12247371673584, 'learning_rate': 8.388888888888888e-06, 'epoch': 2.25}
{'loss': 0.065, 'grad_norm': 12.429498672485352, 'learning_rate': 7.833333333333333e-06, 'epoch': 2.3}
{'loss': 0.0599, 'grad_norm': 1.7471266984939575, 'learning_rate': 7.27777777777777

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.28701451420783997, 'eval_accuracy': 0.927375, 'eval_precision': 0.9321982365460626, 'eval_recall': 0.8954439252336449, 'eval_f1': 0.913451511991658, 'eval_roc_auc': 0.9806868666161526, 'eval_runtime': 16.6193, 'eval_samples_per_second': 481.369, 'eval_steps_per_second': 15.043, 'epoch': 3.0}
{'train_runtime': 960.1935, 'train_samples_per_second': 99.98, 'train_steps_per_second': 3.124, 'train_loss': 0.1650975329875946, 'epoch': 3.0}


TrainOutput(global_step=3000, training_loss=0.1650975329875946, metrics={'train_runtime': 960.1935, 'train_samples_per_second': 99.98, 'train_steps_per_second': 3.124, 'total_flos': 6314665328640000.0, 'train_loss': 0.1650975329875946, 'epoch': 3.0})

In [15]:
# 9) 최종 테스트셋 성능 확인 (evaluate())
final_metrics = trainer.evaluate()
print("\n📊 Final Test Metrics:")
for key, value in final_metrics.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

  0%|          | 0/250 [00:00<?, ?it/s]


📊 Final Test Metrics:
  eval_loss: 0.2096
  eval_accuracy: 0.9283
  eval_precision: 0.9321
  eval_recall: 0.8978
  eval_f1: 0.9146
  eval_roc_auc: 0.9809
  eval_runtime: 16.5926
  eval_samples_per_second: 482.1410
  eval_steps_per_second: 15.0670
  epoch: 3.0000


In [16]:
# 10) 모델 저장
SAVE_DIR = os.path.join(OUTPUT_DIR, "best_model")

# 디렉토리 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# Trainer가 현재 가지고 있는 모델(=best model)을 저장
trainer.save_model(SAVE_DIR)

# 토크나이저도 함께 저장 (나중에 예측 시 필요)
tokenizer.save_pretrained(SAVE_DIR)

print("✅ 모델 저장 완료!")
print(f"📁 저장 경로: {SAVE_DIR}")

✅ 모델 저장 완료!
📁 저장 경로: ./kcelectra_output\best_model
